In [1]:
# 🔧 Instalar solo lo esencial
!pip install -q ultralytics opencv-python-headless

# 📦 Librerías
from ultralytics import YOLO
from tensorflow.keras import layers, models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np, os, cv2, shutil, random, torch

# 📁 Rutas principales

drive = "/content/drive/MyDrive"
yolo_dir = f"{drive}/yolov8/images"
cnn_dir = f"{drive}/your_cnn_data"
yaml_path = f"{yolo_dir}/data.yaml"
img_path = f"{yolo_dir}/test/images/your-image.jpg"
cnn_weights = f"{drive}/Model_Checkpoints/best_model.h5"

# ✅ Verificar GPU
print("¿GPU disponible?:", torch.cuda.is_available())

# ✅ Submuestreo (máx. 2000 imágenes)
def submuestreo_yolo(origen_img, origen_lbl, destino_img, destino_lbl, max_imgs=2000):
    os.makedirs(destino_img, exist_ok=True)
    os.makedirs(destino_lbl, exist_ok=True)
    files = random.sample(os.listdir(origen_img), min(max_imgs, len(os.listdir(origen_img))))
    for f in files:
        shutil.copy(f"{origen_img}/{f}", f"{destino_img}/{f}")
        lbl = f.replace(".jpg", ".txt")
        if os.path.exists(f"{origen_lbl}/{lbl}"):
            shutil.copy(f"{origen_lbl}/{lbl}", f"{destino_lbl}/{lbl}")

submuestreo_yolo(
    f"{yolo_dir}/train/images",
    f"{yolo_dir}/train/labels",
    "/content/dataset_reducido/train/images",
    "/content/dataset_reducido/train/labels"
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/664.8 MB 107.7 MB/s eta 0:00:06
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# ✅ Crear YAML reducido
with open("/content/dataset_reducido/data.yaml", "w") as f:
    f.write(f"""
train: /content/dataset_reducido/train/images
val: {yolo_dir}/valid/images
nc: 12
names: ['clase_{i}' for i in range(1, 13)]
""")

# 🔁 Entrenamiento YOLO (versión rápida)
yolo_model = YOLO("yolov8n.pt")
yolo_model.train(
    data="/content/dataset_reducido/data.yaml",
    epochs=2,
    imgsz=320,
    batch=16,
    cache=True,
    freeze=10,
    name="pest_detector"
)

# 🧠 Convertir YOLO a carpetas por clase (para la CNN)
def yolo_to_cnn(origen, destino):
    for split in ['train', 'valid']:
        src = f"{origen}/{split}/images"
        dst = f"{destino}/{split}"
        os.makedirs(dst, exist_ok=True)
        for f in os.listdir(src):
            if f.lower().endswith(('.jpg', '.png')):
                clase = f.split('-')[0]
                clase_dir = f"{dst}/{clase}"
                os.makedirs(clase_dir, exist_ok=True)
                shutil.copy(f"{src}/{f}", f"{clase_dir}/{f}")

yolo_to_cnn(yolo_dir, cnn_dir)

# 🧠 Modelo CNN simple
def crear_cnn(num_classes):
    inputs = Input(shape=(128, 128, 3))
    x = layers.Conv2D(16, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    return models.Model(inputs, layers.Dense(num_classes, activation='softmax')(x))

# 🧪 Generadores de datos
gen = ImageDataGenerator(rescale=1./255)
train = gen.flow_from_directory(f"{cnn_dir}/train", target_size=(128, 128), batch_size=32, class_mode='categorical')
val = gen.flow_from_directory(f"{cnn_dir}/valid", target_size=(128, 128), batch_size=32, class_mode='categorical')

# ✅ Compilar y entrenar CNN
cnn_model = crear_cnn(train.num_classes)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint(cnn_weights, monitor='val_accuracy', save_best_only=True, mode='max')
cnn_model.fit(train, epochs=5, validation_data=val, callbacks=[checkpoint])




NameError: name 'i' is not defined

In [ ]:
# 🔍 Predicción con YOLO + clasificación CNN
def detectar_y_clasificar(img_path, yolo_model_path, cnn_model_path):
    yolo = YOLO(yolo_model_path)
    results = yolo.predict(source=img_path, conf=0.4, show=True)

    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    cnn = crear_cnn(num_classes=12)
    cnn.load_weights(cnn_model_path)

    for i, box in enumerate(results[0].boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)
        crop = image[y1:y2, x1:x2]
        if crop.size == 0: continue
        resized = cv2.resize(crop, (128, 128))
        input_arr = np.expand_dims(resized / 255.0, axis=0)
        pred = cnn.predict(input_arr, verbose=0)
        clase, conf = np.argmax(pred), np.max(pred)
        print(f"Detección {i+1}: Clase {clase} - Confianza {conf:.2f}")

detectar_y_clasificar(img_path, "runs/detect/pest_detector/weights/best.pt", cnn_weights)